Note: needed to make the rooki env a kernel in ECMWF Jupyter Lab. Followed [this page](https://confluence.ecmwf.int/display/UDOC/Using+your+own+conda+and+virtual+environments+%28venv%29+in+Jupyter+notebooks) 

In [1]:
!pip install rooki

Had error: DuplicateOptionError: While reading from '<dict>': option '__lmod_ref_count_gcc_version' in section 'DEFAULT' already exists

Try to find this bit in the rooki config files and comment out?

In [3]:
import os
os.environ['ROOK_URL'] = 'http://rook.dkrz.de/wps'
import time
from rooki import rooki
from rooki import operators as ops

DuplicateOptionError: While reading from '<dict>': option '__lmod_ref_count_gcc_version' in section 'DEFAULT' already exists

In [6]:
print(os.environ.keys())

KeysView(environ({'JUPYTERHUB_OAUTH_CLIENT_ALLOWED_SCOPES': '[]', 'JUPYTERHUB_CLIENT_ID': 'jupyterhub-user-cxcg', '_EC_PRINT_INFO': '1', '__LMOD_REF_COUNT_gcc_VERSION': '8.5.0:1', 'ECMWF_TOOLBOX_DIR': '/usr/local/apps/ecmwf-toolbox/2024.09.0.0/GNU/8.5', 'SLURM_NODEID': '0', 'SLURM_TASK_PID': '1747340', 'gcc_DIR': '/usr', 'EC_VERBOSE': '0', 'HPCPERM': '/ec/res4/hpcperm/cxcg', '__LMOD_REF_COUNT_PATH': '/usr/local/apps/node/20.11.0/bin:1;/usr/local/apps/python3/3.11.10-01/bin:1;/usr/local/apps/pycharm/2023.3.3/bin:1;/usr/local/apps/ecmwf-toolbox/2024.09.0.0/GNU/8.5/bin:1;/usr/local/bin:1;/usr/bin:1;/usr/local/sbin:1;/usr/sbin:1;/home/cxcg/bin:1', '_ModuleTable002_': 'ID0gImVjbXdmLXRvb2xib3gvMjAyNC4wOS4wLjAiLAp3ViA9ICJeMDAwMDIwMjQuMDAwMDAwMDA5Lip6ZmluYWwiLAp9LApnY2MgPSB7CmZuID0gIi91c3IvbG9jYWwvYXBwcy9tb2R1bGVmaWxlcy9sbW9kL2NvbXBpbGVycy9nY2MvOC41LjAubHVhIiwKZnVsbE5hbWUgPSAiZ2NjLzguNS4wIiwKbG9hZE9yZGVyID0gMSwKcHJvcFQgPSB7fSwKc3RhY2tEZXB0aCA9IDEsCnN0YXR1cyA9ICJhY3RpdmUiLAp1c2VyTmFtZSA9ICJnY2M

Try and explore what is in the c3s atlas dataset, before following the ERA5 subsetting example. James uses intake directly for this.

Need to copy github link to the intake catalog file in question.

In [5]:
import intake

In [6]:
# cat_url = "https://raw.githubusercontent.com/cp4cds/c3s_34g_manifests/master/intake/catalogs/c3s.yaml"

cat_url = "https://raw.githubusercontent.com/cp4cds/c3s_34g_manifests/master/intake/catalogs/c3s.yaml"
cat = intake.open_catalog(cat_url)

How to explore cat without knowing what is in there?

In [7]:
cat

c3s:
  args:
    path: https://raw.githubusercontent.com/cp4cds/c3s_34g_manifests/master/intake/catalogs/c3s.yaml
  description: ''
  driver: intake.catalog.local.YAMLFileCatalog
  metadata: {}


In [8]:
print(list(cat))

['c3s-cmip5', 'c3s-cmip5-daily-pressure-level', 'c3s-cmip5-daily-single-level', 'c3s-cmip5-monthly-pressure-level', 'c3s-cmip5-monthly-single-level', 'c3s-cmip6', 'c3s-cmip6-decadal', 'c3s-cordex', 'c3s-ipcc-atlas', 'c3s-cica-atlas']


In [9]:
df = cat['c3s-cica-atlas'].read()
df.iloc[0]

ds_id                     c3s-cica-atlas.pr.CMIP5.historical.mon.v1
version                                                          v1
path              CMIP5/historical/pr_CMIP5_historical_mon_18500...
size                                                            NaN
project                                                       CMIP5
domain                                                       Global
experiment                                               historical
time_frequency                                                  mon
variable                                                         pr
start_time                                      1850-01-01T00:00:00
end_time                                        2005-12-31T00:00:00
bbox                                                            NaN
level                                                           NaN
url               https://data.mips.climate.copernicus.eu/thredd...
Name: 0, dtype: object

Now look within the source

In [10]:
df

,ds_id,version,path,size,project,domain,experiment,time_frequency,variable,start_time,end_time,bbox,level,url
0,c3s-cica-atlas.pr.CMIP5.historical.mon.v1,v1,CMIP5/historical/pr_CMIP5_historical_mon_18500...,NaN,CMIP5,Global,historical,mon,pr,1850-01-01T00:00:00,2005-12-31T00:00:00,NaN,NaN,https://data.mips.climate.copernicus.eu/thredd...
1,c3s-cica-atlas.prsn.CMIP5.historical.mon.v1,v1,CMIP5/historical/prsn_CMIP5_historical_mon_185...,NaN,CMIP5,Global,historical,mon,prsn,1850-01-01T00:00:00,2005-12-31T00:00:00,NaN,NaN,https://data.mips.climate.copernicus.eu/thredd...
2,c3s-cica-atlas.rx1day.CMIP5.historical.mon.v1,v1,CMIP5/historical/rx1day_CMIP5_historical_mon_1...,NaN,CMIP5,Global,historical,mon,rx1day,1850-01-01T00:00:00,2005-12-31T00:00:00,NaN,NaN,https://data.mips.climate.copernicus.eu/thredd...
3,c3s-cica-atlas.rx5day.CMIP5.historical.mon.v1,v1,CMIP5/historical/rx5day_CMIP5_historical_mon_1...,NaN,CMIP5,Global,historical,mon,rx5day,1850-01-01T00:00:00,2005-12-31T00:00:00,NaN,NaN,https://data.mips.climate.copernicus.eu/thredd...
4,c3s-cica-atlas.sfcwind.CMIP5.historical.mon.v1,v1,CMIP5/historical/sfcwind_CMIP5_historical_mon_...,NaN,CMIP5,Global,historical,mon,sfcwind,1850-01-01T00:00:00,2005-12-31T00:00:00,NaN,NaN,https://data.mips.climate.copernicus.eu/thredd...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407,c3s-cica-atlas.tx40.ERA5.mon.v1,v1,ERA5/tx40_ERA5_mon_194001-202212.nc,NaN,ERA5,Global,NaN,mon,tx40,1940-01-01T00:00:00,2022-12-31T00:00:00,NaN,NaN,https://data.mips.climate.copernicus.eu/thredd...
408,c3s-cica-atlas.txx.ERA5.mon.v1,v1,ERA5/txx_ERA5_mon_194001-202212.nc,NaN,ERA5,Global,NaN,mon,txx,1940-01-01T00:00:00,2022-12-31T00:00:00,NaN,NaN,https://data.mips.climate.copernicus.eu/thredd...
409,c3s-cica-atlas.tx.ERA5.mon.v1,v1,ERA5/tx_ERA5_mon_194001-202212.nc,NaN,ERA5,Global,NaN,mon,tx,1940-01-01T00:00:00,2022-12-31T00:00:00,NaN,NaN,https://data.mips.climate.copernicus.eu/thredd...
410,c3s-cica-atlas.siconc.ORAS5.mon.v1,v1,ORAS5/siconc_ORAS5_mon_195801-201412.nc,NaN,ORAS5,Global,NaN,mon,siconc,1958-01-01T00:00:00,2014-12-31T00:00:00,NaN,NaN,https://data.mips.climate.copernicus.eu/thredd...


Use the dataset ID to filter/search

In [11]:
era5_df = df[df['project'] == 'ERA5']
era5_df

,ds_id,version,path,size,project,domain,experiment,time_frequency,variable,start_time,end_time,bbox,level,url
361,c3s-cica-atlas.cdd.ERA5.yr.v1,v1,ERA5/cdd_ERA5_yr_1940-2022.nc,NaN,ERA5,Global,NaN,yr,cdd,1940-01-01T00:00:00,2022-12-31T00:00:00,NaN,NaN,https://data.mips.climate.copernicus.eu/thredd...
363,c3s-cica-atlas.clt.ERA5.mon.v1,v1,ERA5/clt_ERA5_mon_194001-202212.nc,NaN,ERA5,Global,NaN,mon,clt,1940-01-01T00:00:00,2022-12-31T00:00:00,NaN,NaN,https://data.mips.climate.copernicus.eu/thredd...
364,c3s-cica-atlas.cd.ERA5.yr.v1,v1,ERA5/cd_ERA5_yr_1940-2022.nc,NaN,ERA5,Global,NaN,yr,cd,1940-01-01T00:00:00,2022-12-31T00:00:00,NaN,NaN,https://data.mips.climate.copernicus.eu/thredd...
365,c3s-cica-atlas.evspsbl.ERA5.mon.v1,v1,ERA5/evspsbl_ERA5_mon_194001-202212.nc,NaN,ERA5,Global,NaN,mon,evspsbl,1940-01-01T00:00:00,2022-12-31T00:00:00,NaN,NaN,https://data.mips.climate.copernicus.eu/thredd...
366,c3s-cica-atlas.hd.ERA5.yr.v1,v1,ERA5/hd_ERA5_yr_1940-2022.nc,NaN,ERA5,Global,NaN,yr,hd,1940-01-01T00:00:00,2022-12-31T00:00:00,NaN,NaN,https://data.mips.climate.copernicus.eu/thredd...
367,c3s-cica-atlas.fd.ERA5.mon.v1,v1,ERA5/fd_ERA5_mon_194001-202212.nc,NaN,ERA5,Global,NaN,mon,fd,1940-01-01T00:00:00,2022-12-31T00:00:00,NaN,NaN,https://data.mips.climate.copernicus.eu/thredd...
389,c3s-cica-atlas.mrro.ERA5.mon.v1,v1,ERA5/mrro_ERA5_mon_194001-202212.nc,NaN,ERA5,Global,NaN,mon,mrro,1940-01-01T00:00:00,2022-12-31T00:00:00,NaN,NaN,https://data.mips.climate.copernicus.eu/thredd...
390,c3s-cica-atlas.mrsos.ERA5.mon.v1,v1,ERA5/mrsos_ERA5_mon_194001-202212.nc,NaN,ERA5,Global,NaN,mon,mrsos,1940-01-01T00:00:00,2022-12-31T00:00:00,NaN,NaN,https://data.mips.climate.copernicus.eu/thredd...
391,c3s-cica-atlas.pr.ERA5.mon.v1,v1,ERA5/pr_ERA5_mon_194001-202212.nc,NaN,ERA5,Global,NaN,mon,pr,1940-01-01T00:00:00,2022-12-31T00:00:00,NaN,NaN,https://data.mips.climate.copernicus.eu/thredd...
392,c3s-cica-atlas.rlds.ERA5.mon.v1,v1,ERA5/rlds_ERA5_mon_194001-202212.nc,NaN,ERA5,Global,NaN,mon,rlds,1940-01-01T00:00:00,2022-12-31T00:00:00,NaN,NaN,https://data.mips.climate.copernicus.eu/thredd...


Now, request this data using the WPS, but need to fix the above rooki issue.

In [16]:
era5_df.iloc[0].ds_id

'c3s-cica-atlas.cdd.ERA5.yr.v1'

In [ ]:
# run subset on c3s-cmip5 dataset with time selection
response = rooki.subset(
  collection=era5_df.iloc[0].ds_id)

In [ ]:
# successful?
response.ok
# show links to result files
response.download_urls()